In [1]:
import pandas as pd
import re
import time
import nltk
#nltk.download('stopwords')
from nltk.corpus import sentiwordnet as swn
from nltk.tag import pos_tag,map_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import itertools
import emoji

In [2]:
filepath=r"C:\Aditi's work\Tweet_data\Twitter_extraction\2020-01-21\original_tweet.csv"

pd.options.display.max_colwidth = 300
df = pd.read_csv(filepath, encoding="utf8")
#full_text is fine

df_copy = df
print(len(df_copy))
serlis=df_copy.duplicated().tolist()
print(serlis.count(True)) 

serlis=df_copy.duplicated(['original_text']).tolist()
print(serlis.count(True))



20551
0
413


In [3]:
#pip install emoji

In [4]:
#DROPS
#CHOOSE EITHER TO DROP ALL-ROW DUPLICATES OR FULL_TEXT DUPLICATES
df_copy=df_copy.drop_duplicates(['original_text']) #3377 left after this
df_copy=df_copy.reset_index(drop=True)
#df_copy=df_copy.drop(['place','coordinates','geo','id_str'],axis=1)
df_tweets = df_copy.filter(['original_text'], axis=1)
df_tweets.head(30)

,original_text
0,Prices update in $USDT every 1 hour:\n\n$BTC - 8167.20 $\n$ETH - 165.00 $\n$XRP - 0.25 $\n$TRX - 0.0178 $\n$DOGE - 0.00245318 $\n$LTC - 56.00 $\n\n #BTC #ETH #XRP #TRX #DOGE #LTC #Tradesatoshi #Crypto #Trading
1,#Pelicans putting that work in!! #nba #nbapicks #money
2,"Traders who believe complex strategies payoff best have never taken the time to backtest some of the simplest ones.\n\nBasic, easy to identify strategies work amazingly well. Better yet, they offer consistent profit.\n\n#stocks #forex #futures \n\nJoin FREE at: https://t.co/48IEFICQbq https://t...."
3,"$PLCE in Downtrend: its price expected to drop as it breaks its higher Bollinger Band on January 10, 2020. View odds for this and other indicators: https://t.co/KXHXmecjPb #ChildrensPlace #stockmarket #stock #technicalanalysis #money #trading #investing #daytrading #news #today https://t.co/KJG6..."
4,"Before you reach for your debit or credit card, ask yourself if what you're buying is really worth it. #money #finances https://t.co/vXXqMIjZnc"
5,"Hey Lisa Smolls 🧢 thanks for the follow! If you want more #free #trading #signals, visit https://t.co/AP9gDwl1kT"
6,Bitcoin price index https://t.co/o7UcHJUhC6 #USD #EUR #CNY #GBP #RUB https://t.co/JETNrZscdI
7,"Closed Buy 0.1 Lots EURUSD 1.10846 for +3.3 pips, total for today +3.3 pips #ForexSignals #FX https://t.co/UUJ1YbmrIt"
8,"Closed Buy 0.1 Lots EURUSD 1.10846 for +3.2 pips, total for today +3.2 pips #ForexSignals #FX https://t.co/UUJ1YbmrIt"
9,CFxMLRS update: QTUM sell signal\nCFQTUM 1.917\n\n#QTUM #CFx #MachineLearning #Finance #Cryptocurrency #Trading #Quant https://t.co/pTX6wjm5Lh


In [5]:
# emoji_filepath=r"C:\Aditi's work\Tweet_data\Sentiment_scoring\emojis_clean.csv"
# emojis_dict=pd.read_csv(emoji_filepath, index_col=0, squeeze=True).to_dict()

In [6]:



# source: https://en.wikipedia.org/wiki/List_of_emoticons
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

#CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }


def clean_df(df_tweets):
    
    
    # BASIC CLEANING FUNCTION
    for i in range(len(df_tweets)):
        txt = df_tweets.loc[i]["original_text"]
        
        txt = txt.lower()
        
        txt=re.sub(r'@[A-Z0-9a-z_:]+','',txt)#username-tags
        txt = re.sub('https?://[A-Za-z0-9./]+','',txt)#URLs
#         txt=re.sub(r'#([^\s]+)', r'\1', txt)# remove the # in #hashtag
        txt = ''.join(''.join(s)[:2] for _, s in itertools.groupby(txt)) #Standardizing words eg convert happyyyy to happy
        
        #remove numbers
#         txt=re.sub(r'^[0-9]','', txt)
        
        # Dealing with contractions
        CONTRACTIONS = load_dict_contractions()
        txt = txt.replace("’","'")
        words = txt.split()
        reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
        txt = " ".join(reformed)
        
        # Dealing with emoticons
        SMILEY = load_dict_smileys()  
        words = txt.split()
        reformed = [SMILEY[word] if word in SMILEY else word for word in words]
        txt = " ".join(reformed)
        
        # Dealing with emojis
        txt = emoji.demojize(txt)

        txt = txt.replace(":"," ")
        txt = ' '.join(txt.split())
        
        
        
        df_tweets.at[i,"original_text"]=txt
    return df_tweets
        
df_tweets_clean=clean_df(df_tweets)


In [7]:
df_tweets_clean.head(60)
df_tweets_clean.to_csv (r"C:\Aditi's work\Tweet_data\Sentiment_scoring\cleaned_tweets.csv", index = None, header=True)

In [12]:
import codecs

def read_sentiment_hashtags():
    with codecs.open("C:\Aditi's work\Tweet_data\Sentiment_scoring\\NRC-Hashtag-Sentiment-Lexicon-v0.1\\unigrams-pmilexicon\\unigrams-pmilexicon.txt", 'r', encoding='utf8') as f:
        hashtags = f.read().splitlines()
    sentiment_hashs = dict()
    for hashtag in hashtags:
        l = hashtag.split('\t')
        sentiment_hashs[l[0]] = l[1]
    return sentiment_hashs

In [13]:
#POS-TAGGING AND SENTIMENT SCORE
#from nltk import punkt

pstem = PorterStemmer()
lem = WordNetLemmatizer()
stop_words = stopwords.words('english')

sent_score_unigram=read_sentiment_hashtags()

def sentiment_score(df_tweets_clean):
    li_swn=[]
    li_swn_pos=[]
    li_swn_neg=[]
    missing_words=[]
    for i in range(len(df_tweets_clean.index)):
        text = df_tweets_clean.loc[i]['original_text']
        tokens = nltk.word_tokenize(text)
#         tagged_sent = pos_tag(tokens)
#         store_it = [(word, map_tag('en-ptb', 'universal', tag)) for word, tag in tagged_sent]
        #print("Tagged Parts of Speech:",store_it)

        pos_total=0
        neg_total=0
        this_word_pos=0
        this_word_neg=0
        for word in tokens:
            try:
                if float(sent_score_unigram[word])>0:
                    this_word_pos=float(sent_score_unigram[word])
                else:
                    this_word_neg=float(sent_score_unigram[word])
                #print(word,tag,':',this_word_pos,this_word_neg)
            except Exception as e:
                lem_word = lem.lemmatize(word)
                # Checking if there's a possiblity of lemmatized word be accepted into SWN corpus
                try:
                    if float(sent_score_unigram[lem_word])>0:
                        this_word_pos=float(sent_score_unigram[lem_word])
                    else:
                        this_word_neg=float(sent_score_unigram[lem_word])
                except Exception as e:
                    stem_word = pstem.stem(word)
                    # Checking if there's a possiblity of lemmatized word be accepted
                    try:
                        if float(sent_score_unigram[stem_word])>0:
                            this_word_pos=float(sent_score_unigram[stem_word])
                        else:
                            this_word_neg=float(sent_score_unigram[stem_word])
                    except:
                        missing_words.append(word) 
                        continue
            pos_total+=this_word_pos
            neg_total+=this_word_neg
        li_swn_pos.append(pos_total)
        li_swn_neg.append(neg_total)

#         if(pos_total!=0 or neg_total!=0):
#             if(abs(pos_total)>abs(neg_total)):
#                 li_swn.append(1)
#             else:
#                 li_swn.append(-1)
#         else:
#             li_swn.append(0)

        li_swn.append((pos_total+neg_total))
    
    print("scoring done ============================================")
            
    # end-of pos-tagging&sentiment
    
    #3-rd for loop    
    #LEMMATIZING,STEMMING,STOP-WORDS
    for i in range(len(df_tweets_clean.index)):
        text = df_tweets_clean.loc[i]['original_text']
        tokens = nltk.word_tokenize(text)
        tokens = [word for word in tokens if word not in stop_words]

#         for j in range(len(tokens)):
#             tokens[j] = lem.lemmatize(tokens[j])
#             tokens[j] = pstem.stem(tokens[j])

        tokens_sent=' '.join(tokens)
        df_tweets_clean.at[i,"original_text"] = tokens_sent
        
    df_tweets_clean.insert(1,"pos_score",li_swn_pos,True)
    df_tweets_clean.insert(2,"neg_score",li_swn_neg,True)
    df_tweets_clean.insert(3,"sent_score",li_swn,True)
    
#     print("missing words:"+ missing_words)
    
    
    
    return df_tweets_clean


In [14]:
st=time.time()
sentiment_score(df_tweets_clean)
end=time.time()
print(end-st)




scoring done ============================================
51.616164684295654


In [15]:
df_tweets_clean.head(50)


,original_text,pos_score,neg_score,sent_score
0,prices update $ usdt every 1 hour $ btc - 8167.20 $ $ eth - 165.00 $ $ xrp - 0.25 $ $ trx - 0.0178 $ $ doge - 0.00245318 $ $ ltc - 56.00 $ # btc # eth # xrp # trx # doge # ltc # tradesatoshi # crypto # trading,9.210,-26.980,-17.770
1,# pelicans putting work ! ! # nba # nbapicks # money,2.129,-8.877,-6.748
2,"traders believe complex strategies payoff best never taken time backtest simplest ones . basic , easy identify strategies work amazingly well . better yet , offer consistent profit . # stocks # forex # futures join free",17.161,-19.363,-2.202
3,"$ plce downtrend price expected drop breaks higher bollinger band january 10 , 2020. view odds indicators # childrensplace # stockmarket # stock # technicalanalysis # money # trading # investing # daytrading # news # today",20.579,-10.104,10.475
4,"reach debit credit card , ask buying really worth . # money # finances",3.797,-8.892,-5.095
5,"hey lisa smolls billed_cap thanks follow ! want # free # trading # signals , visit",5.440,-2.855,2.585
6,bitcoin price index # usd # eur # cny # gbp # rub,0.478,-6.364,-5.886
7,"closed buy 0.1 lots eurusd 1.10846 +3.3 pips , total today +3.3 pips # forexsignals # fx",3.725,-8.092,-4.367
8,"closed buy 0.1 lots eurusd 1.10846 +3.2 pips , total today +3.2 pips # forexsignals # fx",3.725,-8.092,-4.367
9,cfxmlrs update qtum sell signal cfqtum 1.917 # qtum # cfx # machinelearning # finance # cryptocurrency # trading # quant,1.884,-3.001,-1.117


In [16]:
sentwordnet_score_csv = df_tweets_clean.to_csv (r"C:\Aditi's work\Tweet_data\Sentiment_scoring\nrc-unigram_score_csv_netscore.csv", index = None, header=True)